In [1]:
import numpy as np
import torch
 
loss = torch.nn.CrossEntropyLoss()#softmax 포함
 
Y=torch.LongTensor([0]) # 64bit integer, class label
Y.requires_grad=False

Y_pred1 = torch.Tensor([[2.0,1.0,0.1]])# default tensor type
Y_pred2 = torch.Tensor([[0.5,2.0,0.3]])
 
l1 = loss(Y_pred1,Y)
l2 = loss(Y_pred2,Y)
 
print(l1,l2)

tensor(0.4170) tensor(1.8406)


In [2]:
loss = torch.nn.CrossEntropyLoss()
 
 
Y=torch.LongTensor([2,0,1])
Y.requires_grad=False
 
Y_pred1 = torch.Tensor([[2.0,1.0,0.1],
                       [1.0,1.4,0.1],
                       [1.1,0.2,3.1]])
Y_pred2 = torch.Tensor([[0.5,1.0,2.5],
                       [2.2,1.1,0.3],
                       [1.5,2.0,1.3]])
 
l1 = loss(Y_pred1,Y)
l2 = loss(Y_pred2,Y)
 
print(l1,l2)

tensor(2.1518) tensor(0.4812)


In [12]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
 
#마법의 단어 device~
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 64
 
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)
 
test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())
 
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
 
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)
class NNModel(torch.nn.Module):
     
    def __init__(self):
        super(NNModel,self).__init__()
        self.l1 = nn.Linear(784,520)
        self.l2 = nn.Linear(520,320)
        self.l3 = nn.Linear(320,240)
        self.l4 = nn.Linear(240,120)
        self.l5 = nn.Linear(120,10)
         
    def forward(self,x):
        # input data : ( n , 1 , 28 , 28 )
        x = x.view(-1,784) # Flatten : ( n , 784 )
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)

model = NNModel()
 
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.01, momentum = 0.5)
 
model= model.to(device)
 
def train(epoch):
    model.train()
    for batch_idx,(data,target) in enumerate(train_loader):
         
        data = data.to(device)
        target = target.to(device)
         
        output = model(data)
 
        optimizer.zero_grad()
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
         
        if batch_idx%50==0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),100. * batch_idx / len(train_loader), loss.item()))
            #loss.data[0] -> loss.item()
             
def test():
    model.eval()
    test_loss=0
    correct=0
    for data,target in test_loader:
         
        data = data.to(device)
        target = target.to(device)
         
        output = model(data)
        #model.forward(data) 
        test_loss += criterion(output,target).item()
        #criterion.data[0] -> criterion.item()
        pred = output.data.max(1,keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()
         
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
             
             
for epoch in range(1, 3):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.305351
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.305516
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.296897
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.286401
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.288840
Train Epoch: 1 [16000/60000 (27%)]	Loss: 2.299943
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.291595
Train Epoch: 1 [22400/60000 (37%)]	Loss: 2.287410
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.276941
Train Epoch: 1 [28800/60000 (48%)]	Loss: 2.272712
Train Epoch: 1 [32000/60000 (53%)]	Loss: 2.271921
Train Epoch: 1 [35200/60000 (59%)]	Loss: 2.266979
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.267298
Train Epoch: 1 [41600/60000 (69%)]	Loss: 2.238389
Train Epoch: 1 [44800/60000 (75%)]	Loss: 2.211153
Train Epoch: 1 [48000/60000 (80%)]	Loss: 2.184007
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.076484
Train Epoch: 1 [54400/60000 (91%)]	Loss: 1.970570
Train Epoch: 1 [57600/60000 (96%)]	Loss: 1.597196

Test set: Average loss: 0.0238, Accuracy: 4606/10000 (46%)